#  Cleaning and Combining Data With Pandas


### Overview
In this lesson, students will learn how to use the Pandas library to clean data and combine multiple DataFrames.

### Learning Objectives
* Use Pandas to handle missing or problematic data values.
* Identify appropriate cleaning strategies for specific types of data.
* Use groupby() and JOIN statements to combine data with Pandas.
* Create insights from data by splitting and combining data segments.


In [1]:
import pandas as pd
df = pd.read_csv('data/ufo2.csv')
df.head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1,1931
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1,1933


In [2]:
df.dtypes

City               object
Colors Reported    object
Shape Reported     object
State              object
Time               object
sightings           int64
Year                int64
dtype: object

In [3]:
# isnull converts values into True if NULL, False otherwise
df.isnull().head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year
0,False,True,False,False,False,False,False
1,False,True,False,False,False,False,False
2,False,True,False,False,False,False,False
3,False,True,False,False,False,False,False
4,False,True,False,False,False,False,False


In [4]:
# we can then use .sum() to count up the number of NULL values
df.isnull().sum()

City                  47
Colors Reported    63509
Shape Reported         0
State                  0
Time                   0
sightings              0
Year                   0
dtype: int64

##  Options for Missing Values

### Remove them

In [5]:
# Remove rows with all cols missing
print(df.shape)
df.dropna(inplace=False).shape

(80543, 7)


(17027, 7)

In [6]:
# Remove rows with one col missing
print(df.shape)
df["City"].dropna(inplace=False).shape

(80543, 7)


(80496,)

In [7]:
# Remove rows with only some cols missing
print(df.shape)
df.dropna(subset=["City", "Shape Reported"], inplace=False).shape

(80543, 7)


(80496, 7)

### Fill with arbitrary value

In [8]:
df = pd.read_csv('data/ufo2.csv')
print(df.isnull().sum().head())

City                  47
Colors Reported    63509
Shape Reported         0
State                  0
Time                   0
dtype: int64


In [9]:
df["City"].fillna("Austin", inplace=True)
print(df.isnull().sum().head())

City                   0
Colors Reported    63509
Shape Reported         0
State                  0
Time                   0
dtype: int64


In [10]:
df.fillna({"City":'Austin', "Shape Reported":'donut'}, inplace=True)
print(df.isnull().sum().head())

City                   0
Colors Reported    63509
Shape Reported         0
State                  0
Time                   0
dtype: int64


## Fill with computed value

In [11]:
df = pd.read_csv('data/ufo2.csv')
print(df.isnull().sum().head())

City                  47
Colors Reported    63509
Shape Reported         0
State                  0
Time                   0
dtype: int64


In [12]:
df["sightings"].fillna(df["sightings"].mean(), inplace=True)

In [13]:
top = df["Shape Reported"].describe().top
print(top)
df["Shape Reported"].fillna(top, inplace=True)

LIGHT


##  More Options for Data Cleaning

### Replace specific values

In [14]:
df.head(3)

,City,Colors Reported,Shape Reported,State,Time,sightings,Year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931


In [15]:
# replaces all appearances of
df.replace("TRIANGLE", "SpaceShip", inplace=True)
df.head(3)

,City,Colors Reported,Shape Reported,State,Time,sightings,Year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931


## Fill with interpolated values

In [16]:
# given [0,1,NA,3] as a series...
df['Colors Reported'].interpolate()

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
80538         NaN
80539         NaN
80540    RED BLUE
80541         RED
80542         NaN
Name: Colors Reported, Length: 80543, dtype: object

# Cleaning and Formatting Data
 

###  Modifying Series Within DataFrames

In [17]:
df.dtypes

City               object
Colors Reported    object
Shape Reported     object
State              object
Time               object
sightings           int64
Year                int64
dtype: object

In [18]:
# turn an int to a string
df['string-year']=df['Year'].apply(str)

In [19]:
# Many of our cleaning operations involve applying an operation to a Series
df['int-year'] = pd.to_numeric(df['string-year'])

In [20]:
df[['Time', 'string-year', 'int-year']].head()

,Time,string-year,int-year
0,1930-06-01 22:00:00,1930,1930
1,1930-06-30 20:00:00,1930,1930
2,1931-02-15 14:00:00,1931,1931
3,1931-06-01 13:00:00,1931,1931
4,1933-04-18 19:00:00,1933,1933


In [21]:
#  This can also be used to create new columns based on existing data:
df['star-trek-stardate'] = df['int-year']+350
df['star-trek-stardate'].head()

0    2280
1    2280
2    2281
3    2281
4    2283
Name: star-trek-stardate, dtype: int64

##  Handling Dates and Times

In [22]:
df.head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930,1930,1930,2280
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930,1930,1930,2280
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931,1931,1931,2281
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1,1931,1931,1931,2281
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1,1933,1933,1933,2283


In [23]:
# the date column is a string
df['Time'].dtypes

dtype('O')

In [24]:
df['newdate']=pd.to_datetime(df['Time'])
df[['Time', 'newdate']].head()

,Time,newdate
0,1930-06-01 22:00:00,1930-06-01 22:00:00
1,1930-06-30 20:00:00,1930-06-30 20:00:00
2,1931-02-15 14:00:00,1931-02-15 14:00:00
3,1931-06-01 13:00:00,1931-06-01 13:00:00
4,1933-04-18 19:00:00,1933-04-18 19:00:00


In [25]:
df['Month']=df['newdate'].dt.month
df[['Time', 'newdate', 'Month']].head()

,Time,newdate,Month
0,1930-06-01 22:00:00,1930-06-01 22:00:00,6
1,1930-06-30 20:00:00,1930-06-30 20:00:00,6
2,1931-02-15 14:00:00,1931-02-15 14:00:00,2
3,1931-06-01 13:00:00,1931-06-01 13:00:00,6
4,1933-04-18 19:00:00,1933-04-18 19:00:00,4


##  Anatomy of a TimeStamp

In [26]:
moment_in_time = pd.Timestamp("19890602T07:43:55")

In [27]:
# now call the elements
moment_in_time.year

1989

In [28]:
moment_in_time.month

6

##  apply()

In [29]:
# Let's make some fake data

In [42]:
df['dollars']='$'
df['year_dollars']=df['dollars']+df['string-year']
df.head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,Month,dollars,year_dollars
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930,1930,1930,2280,1930-06-01 22:00:00,6,$,$1930
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930,1930,1930,2280,1930-06-30 20:00:00,6,$,$1930
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931,1931,1931,2281,1931-02-15 14:00:00,2,$,$1931
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1,1931,1931,1931,2281,1931-06-01 13:00:00,6,$,$1931
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1,1933,1933,1933,2283,1933-04-18 19:00:00,4,$,$1933


In [43]:
df['year_dollars']

0        $1930
1        $1930
2        $1931
3        $1931
4        $1933
         ...  
80538    $2014
80539    $2014
80540    $2014
80541    $2014
80542    $2014
Name: year_dollars, Length: 80543, dtype: object

In [44]:
# There is also the apply() method
def strip_dollar_sign(str):
    return str.replace("$", "")

In [48]:
df['clean-year'] = df['year_dollars'].apply(strip_dollar_sign)
df[['year_dollars', 'clean-year']].head()

,year_dollars,clean-year
0,$1930,1930
1,$1930,1930
2,$1931,1931
3,$1931,1931
4,$1933,1933


In [49]:
df.head(2)

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,Month,dollars,year_dollars,clean-year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930,1930,1930,2280,1930-06-01 22:00:00,6,$,$1930,1930
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930,1930,1930,2280,1930-06-30 20:00:00,6,$,$1930,1930


In [59]:
# If you need to apply a function to each row by row, set axis=1 
def star_wars_date(row):
    return row['star-trek-stardate'] * row['Month'] + row['int-year'] - 20000

In [60]:
df['star-wars-year'] = df.apply(star_wars_date, axis=1)
df[['star-wars-year', 'star-trek-stardate', 'Year']].head()

,star-wars-year,star-trek-stardate,Year
0,-4390,2280,1930
1,-4390,2280,1930
2,-13507,2281,1931
3,-4383,2281,1931
4,-8935,2283,1933


##  Addressing Duplicates

In [68]:
df.head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,Month,dollars,year_dollars,clean-year,star-wars-year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930,1930,1930,2280,1930-06-01 22:00:00,6,$,$1930,1930,-4390
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930,1930,1930,2280,1930-06-30 20:00:00,6,$,$1930,1930,-4390
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931,1931,1931,2281,1931-02-15 14:00:00,2,$,$1931,1931,-13507
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1,1931,1931,1931,2281,1931-06-01 13:00:00,6,$,$1931,1931,-4383
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1,1933,1933,1933,2283,1933-04-18 19:00:00,4,$,$1933,1933,-8935


In [73]:
# how many rows are duplicates?
df.duplicated().sum()

609

In [74]:
# take a look at a few duplicated rows
df[df.duplicated(keep=False)].head(4)

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,Month,dollars,year_dollars,clean-year,star-wars-year
21,NaN,NaN,LIGHT,LA,1943-08-15 00:00:00,2,1943,1943,1943,2293,1943-08-15 00:00:00,8,$,$1943,1943,287
22,NaN,NaN,LIGHT,LA,1943-08-15 00:00:00,2,1943,1943,1943,2293,1943-08-15 00:00:00,8,$,$1943,1943,287
194,Miami,NaN,DISK,FL,1952-06-30 21:00:00,9,1952,1952,1952,2302,1952-06-30 21:00:00,6,$,$1952,1952,-4236
195,Miami,NaN,DISK,FL,1952-06-30 21:00:00,9,1952,1952,1952,2302,1952-06-30 21:00:00,6,$,$1952,1952,-4236


In [75]:
# how about a subset?
df.duplicated(subset=['City','State']).sum()

62258

In [77]:
# get rid of them
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(80543, 16)
(79934, 16)


In [80]:
#  If we want to drop duplicates based on certain columns, we can do that, too:
print(df.shape)
df.drop_duplicates(subset=['City','State'], inplace=True)
print(df.shape)

(79934, 16)
(18285, 16)


## Combining Data
 

###  Aggregating With groupby()

In [86]:
df.head()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,Month,dollars,year_dollars,clean-year,star-wars-year
0,Ithaca,NaN,SpaceShip,NY,1930-06-01 22:00:00,1,1930,1930,1930,2280,1930-06-01 22:00:00,6,$,$1930,1930,-4390
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1,1930,1930,1930,2280,1930-06-30 20:00:00,6,$,$1930,1930,-4390
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1,1931,1931,1931,2281,1931-02-15 14:00:00,2,$,$1931,1931,-13507
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1,1931,1931,1931,2281,1931-06-01 13:00:00,6,$,$1931,1931,-4383
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1,1933,1933,1933,2283,1933-04-18 19:00:00,4,$,$1933,1933,-8935


In [87]:
#  groupby() statements allow us to segment our population to a specific subset 
df.groupby(['Month']).count()

,City,Colors Reported,Shape Reported,State,Time,sightings,Year,string-year,int-year,star-trek-stardate,newdate,dollars,year_dollars,clean-year,star-wars-year
Month,,,,,,,,,,,,,,,
1,1144,233,1145,1145,1145,1145,1145,1145,1145,1145,1145,1145,1145,1145,1145
2,974,203,976,976,976,976,976,976,976,976,976,976,976,976,976
3,1002,170,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002
4,1117,202,1119,1119,1119,1119,1119,1119,1119,1119,1119,1119,1119,1119,1119
5,1210,191,1210,1210,1210,1210,1210,1210,1210,1210,1210,1210,1210,1210,1210
6,2522,329,2526,2526,2526,2526,2526,2526,2526,2526,2526,2526,2526,2526,2526
7,2443,456,2446,2446,2446,2446,2446,2446,2446,2446,2446,2446,2446,2446,2446
8,2039,361,2043,2043,2043,2043,2043,2043,2043,2043,2043,2043,2043,2043,2043
9,1691,311,1691,1691,1691,1691,1691,1691,1691,1691,1691,1691,1691,1691,1691


##  Taking Aggregate Measures

In [88]:
df.groupby('Month')['sightings'].agg(['count', 'mean', 'min', 'max'])

,count,mean,min,max
Month,,,,
1,1145,11.531878,1,100
2,976,9.360656,1,37
3,1002,10.245509,1,76
4,1119,9.687221,1,42
5,1210,9.076860,1,50
6,2526,14.621536,1,56
7,2446,22.911284,1,258
8,2043,14.585414,1,61
9,1691,16.280899,1,124


##  Concatenate

In [91]:
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number']) 
df1

,letter,number
0,a,1
1,b,2


In [92]:
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
df2

,letter,number
0,c,3
1,d,4


In [90]:
df_with_more_rows = pd.concat([df1, df2]) 
df_with_more_rows

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [93]:
df_with_more_columns = pd.concat([df1, df2], axis=1)
df_with_more_columns 

,letter,number,letter,number
0,a,1,c,3
1,b,2,d,4


##  Using merge

In [ ]:
# pd.merge(left_df, right_df, how, left_on, right_on)

In [94]:
# stock names
stockname = pd.DataFrame({'Symbol': ['AMZN', 'MSFT', 'FB', 'AAPL', 'GOOGL'], 'Name': ['Amazon', 'Microsoft', 'Facebook', 'Apple', 'Google']})
stockname

,Symbol,Name
0,AMZN,Amazon
1,MSFT,Microsoft
2,FB,Facebook
3,AAPL,Apple
4,GOOGL,Google


In [95]:
# stock prices.
openprice = pd.DataFrame({'Symbol': ['AAPL', 'MSFT', 'GOOGL', 'FB', 'AMZN'], 'OpenPrice': [217.51, 96.54, 501.3, 51.45, 1703.34]})
openprice

,Symbol,OpenPrice
0,AAPL,217.51
1,MSFT,96.54
2,GOOGL,501.30
3,FB,51.45
4,AMZN,1703.34


In [96]:
# Merge these dataframes.
named_stocks=pd.merge(openprice,  stockname)
named_stocks

,Symbol,OpenPrice,Name
0,AAPL,217.51,Apple
1,MSFT,96.54,Microsoft
2,GOOGL,501.30,Google
3,FB,51.45,Facebook
4,AMZN,1703.34,Amazon


In [97]:
# Create a 3rd dataset of weekly highs
wkhigh = pd.DataFrame({'Symbol': ['FB', 'AMZN', 'AAPL', 'MSFT', 'NFLX'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11, 303.22]})
wkhigh

,Symbol,52wkHigh
0,FB,60.79
1,AMZN,2050.49
2,AAPL,233.47
3,MSFT,110.11
4,NFLX,303.22


In [98]:
# Now merge that with the named stocks.
full_stocks=pd.merge(named_stocks, wkhigh, on='Symbol', how='outer')
# full_stocks=pd.merge(named_stocks, wkhigh, on='Symbol', how='left')
# full_stocks=pd.merge(named_stocks, wkhigh, on='Symbol', how='inner')
# full_stocks=pd.merge(named_stocks, wkhigh, on='Symbol', how='right')
full_stocks

,Symbol,OpenPrice,Name,52wkHigh
0,AAPL,217.51,Apple,233.47
1,MSFT,96.54,Microsoft,110.11
2,GOOGL,501.30,Google,NaN
3,FB,51.45,Facebook,60.79
4,AMZN,1703.34,Amazon,2050.49
5,NFLX,NaN,NaN,303.22
